In [ ]:
# Install necessary libraries in Google Colab
!pip install pandas matplotlib torch ta

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import itertools
import random
import logging
from datetime import datetime
import ta

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(message)s",
    handlers=[
        logging.FileHandler("hyperparameter_testing.log"),
        logging.StreamHandler()
    ]
)

# Load dataset (replace with your actual file path)
data_path = "btcusd_30m.csv"  # Ensure you've uploaded the dataset
data = pd.read_csv(data_path, parse_dates=True, index_col="date")

# Compute technical indicators
# Define the periods we will use for moving averages and RSI
ma_periods = [5, 10, 15, 20, 25, 30]
rsi_periods = [14]

# Calculate SMA and EMA for the defined periods
for period in ma_periods:
    data[f'SMA_{period}'] = data['close'].rolling(window=period).mean()
    data[f'EMA_{period}'] = data['close'].ewm(span=period, adjust=False).mean()

# Calculate RSI for the defined periods
for period in rsi_periods:
    rsi_indicator = ta.momentum.RSIIndicator(close=data['close'], window=period)
    data[f'RSI_{period}'] = rsi_indicator.rsi()

# Handle NaN values resulting from indicator calculations
data.dropna(inplace=True)

# Move data to PyTorch tensors on GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_tensor = {col: torch.tensor(data[col].values, device=device, dtype=torch.float32) for col in data.columns}

# Constants
INITIAL_BALANCE = 250
BROKER_FEE = 0.002  # 0.2%
SLIPPAGE = 0.00005  # 0.005%
RANDOM_SLICES = 10
SLICE_SIZES = [500, 1000, 1500, 2000, 2500, 3000, 5000, 6000, 7000, 8000, 10000, 15000, 20000, 22500, 25000, 30000, len(data)]  # Random sizes for testing

# Define hyperparameters to test
HYPERPARAMETERS = {
    "period": [5, 10, 15],  # Limiting for computational feasibility
    "max_length": [100, 200],
    "threshold_rate": [1, 2],
    "min_tests": [2, 3],
    "ma_type": ["SMA", "EMA"],
    "ma_period": [10, 20],
    "rsi_period": [14],
    "rsi_overbought": [70],
    "rsi_oversold": [30],
    "indicator_threshold": [1, 2]  # Number of indicators that must agree
}

class BreakoutFinder:
    def __init__(self, period=5, max_length=200, threshold_rate=3, min_tests=2,
                 ma_type='SMA', ma_period=14, rsi_period=14,
                 rsi_overbought=70, rsi_oversold=30, indicator_threshold=2):
        self.period = period
        self.max_length = max_length
        self.threshold_rate = threshold_rate / 100
        self.min_tests = min_tests
        self.ma_type = ma_type
        self.ma_period = ma_period
        self.rsi_period = rsi_period
        self.rsi_overbought = rsi_overbought
        self.rsi_oversold = rsi_oversold
        self.indicator_threshold = indicator_threshold  # Number of indicators that must agree

    def find_breakouts(self, data):
        high_max = torch.nn.functional.max_pool1d(
            data['high'].unsqueeze(0).unsqueeze(0),
            kernel_size=self.period,
            stride=1,
            padding=self.period // 2
        ).squeeze()
        low_min = torch.nn.functional.max_pool1d(
            -data['low'].unsqueeze(0).unsqueeze(0),
            kernel_size=self.period,
            stride=1,
            padding=self.period // 2
        ).squeeze() * -1

        breakout_signals = []
        breakdown_signals = []

        for i in range(len(data['high'])):
            high_tests = data['high'][max(0, i - self.max_length):i]
            low_tests = data['low'][max(0, i - self.max_length):i]

            if len(high_tests) >= self.min_tests:
                max_pivot = high_tests.max()
                if data['close'][i] > max_pivot:
                    breakout_signals.append((i, max_pivot.item()))

            if len(low_tests) >= self.min_tests:
                min_pivot = low_tests.min()
                if data['close'][i] < min_pivot:
                    breakdown_signals.append((i, min_pivot.item()))

        return breakout_signals, breakdown_signals

    def apply_trading_strategy(self, data):
        breakout_signals, breakdown_signals = self.find_breakouts(data)
        signal = torch.zeros(len(data['close']), device=device)

        # Generate breakout signals
        breakout_signal_series = torch.zeros(len(data['close']), device=device)
        for idx, _ in breakout_signals:
            breakout_signal_series[idx] = 1
        for idx, _ in breakdown_signals:
            breakout_signal_series[idx] = -1

        # Generate MA crossover signals
        ma_column = f'{self.ma_type}_{self.ma_period}'
        if ma_column in data:
            ma_series = data[ma_column]
        else:
            ma_series = torch.zeros(len(data['close']), device=device)  # default to zeros

        ma_signal_series = torch.zeros(len(data['close']), device=device)
        close_price = data['close']

        # MA crossover signals
        ma_signal_series[1:] = torch.where(
            (close_price[1:] > ma_series[1:]) & (close_price[:-1] <= ma_series[:-1]), 1,
            torch.where(
                (close_price[1:] < ma_series[1:]) & (close_price[:-1] >= ma_series[:-1]), -1, 0
            )
        )

        # Generate RSI signals
        rsi_column = f'RSI_{self.rsi_period}'
        if rsi_column in data:
            rsi_series = data[rsi_column]
        else:
            rsi_series = torch.zeros(len(data['close']), device=device)

        rsi_signal_series = torch.where(
            rsi_series < self.rsi_oversold, 1,
            torch.where(
                rsi_series > self.rsi_overbought, -1, 0
            )
        )

        # Combine signals
        total_signals = breakout_signal_series + ma_signal_series + rsi_signal_series
        signal = torch.where(total_signals >= self.indicator_threshold, 1,
                             torch.where(total_signals <= -self.indicator_threshold, -1, 0))

        return signal

def backtest(data, signals):
    balance = INITIAL_BALANCE
    position = 0
    portfolio = []
    trade_log = []

    for i in range(len(data['close'])):
        signal = signals[i]
        close_price = data['close'][i]

        if signal == 1 and position == 0:
            position = (balance / close_price) * (1 - BROKER_FEE - SLIPPAGE)
            balance = 0
            trade_log.append((i, "BUY", close_price.item(), balance, position))
        elif signal == -1 and position > 0:
            balance = (position * close_price) * (1 - BROKER_FEE - SLIPPAGE)
            position = 0
            trade_log.append((i, "SELL", close_price.item(), balance, position))

        portfolio.append(balance + position * close_price if position else balance)

    return torch.tensor(portfolio, device=device), trade_log

def run_single_test(params):
    (period, max_length, threshold_rate, min_tests,
     ma_type, ma_period, rsi_period, indicator_threshold) = params

    # Ensure we have enough data for the indicators
    max_indicator_period = max(ma_period, rsi_period)
    slice_size = random.choice(SLICE_SIZES)

    # Adjust slice_size if it's too large
    max_possible_slice_size = len(data_tensor['close']) - max_indicator_period
    if slice_size > max_possible_slice_size:
        slice_size = max_possible_slice_size

    if slice_size <= 0:
        # Not enough data to run the test
        logging.warning("Not enough data for the selected slice_size and indicator periods.")
        return params, INITIAL_BALANCE, torch.tensor([INITIAL_BALANCE], device=device), []

    # Adjust start_idx to ensure valid range
    max_start_idx = len(data_tensor['close']) - slice_size
    start_idx = random.randint(max_indicator_period, max_start_idx)

    sliced_data = {key: val[start_idx:start_idx + slice_size] for key, val in data_tensor.items()}

    breakout_finder = BreakoutFinder(
        period=period,
        max_length=max_length,
        threshold_rate=threshold_rate,
        min_tests=min_tests,
        ma_type=ma_type,
        ma_period=ma_period,
        rsi_period=rsi_period,
        indicator_threshold=indicator_threshold
    )

    signals = breakout_finder.apply_trading_strategy(sliced_data)
    portfolio, trade_log = backtest(sliced_data, signals)

    final_value = portfolio[-1].item() if len(portfolio) > 0 else INITIAL_BALANCE
    logging.info(f"Tested params: {params} | Final Portfolio Value: {final_value}")
    return params, final_value, portfolio, trade_log

def run_tests():
    param_combinations = list(itertools.product(
        HYPERPARAMETERS['period'],
        HYPERPARAMETERS['max_length'],
        HYPERPARAMETERS['threshold_rate'],
        HYPERPARAMETERS['min_tests'],
        HYPERPARAMETERS['ma_type'],
        HYPERPARAMETERS['ma_period'],
        HYPERPARAMETERS['rsi_period'],
        HYPERPARAMETERS['indicator_threshold']
    ))

    logging.info(f"Total parameter combinations to test: {len(param_combinations)}")
    results = []

    for i, params in enumerate(param_combinations):
        logging.info(f"Testing combination {i + 1}/{len(param_combinations)}: {params}")
        results.append(run_single_test(params))

    return results

# Execute the hyperparameter testing
logging.info("Executing GPU-accelerated hyperparameter testing...")
start_time = datetime.now()
results = run_tests()
end_time = datetime.now()

# Analyze results
results_df = pd.DataFrame([{
    "params": r[0],
    "final_value": r[1]
} for r in results]).sort_values(by="final_value", ascending=False)

# Extract best params and portfolio
best_params = results_df.iloc[0]['params']
best_portfolio = [r for r in results if r[0] == best_params][0][2].cpu().numpy()

# Plot best portfolio performance
plt.figure(figsize=(14, 7))
plt.plot(best_portfolio, label=f"Best Portfolio (Params: {best_params})")
plt.title("Best Portfolio Performance")
plt.xlabel("Time")
plt.ylabel("Portfolio Value (USD)")
plt.legend()
plt.grid()
plt.show()

# Save results and log completion time
results_df.to_csv("hyperparameter_results.csv", index=False)
logging.info(f"Hyperparameter testing completed in {end_time - start_time}.")
logging.info(f"Best Parameters: {best_params}")
logging.info("Results saved to 'hyperparameter_results.csv'.")
